Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [1]:
%%bash
pip install llama-recipes transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import transformers
print('/'.join(transformers.__file__.split('/')[:-1]) + '/models/llama/convert_llama_weights_to_hf.py')

/jet/home/apatula/.local/lib/python3.10/site-packages/transformers/models/llama/convert_llama_weights_to_hf.py


In [3]:
%%bash

python '/jet/home/apatula/.local/lib/python3.10/site-packages/transformers/models/llama/convert_llama_weights_to_hf.py' --input_dir llama-2-7b --model_size 7B --output_dir models_hf/7B

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Fetching all parameters from the checkpoint at llama-2-7b.
Loading the checkpoint in a Llama model.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 33/33 [00:03<00:00,  9.44it/s]


Saving in the Transformers format.


### Step 1: Load the model

Point model_id to model weight folder

In [4]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id="./models_hf/7B"

tokenizer = LlamaTokenizer.from_pretrained(model_id)

model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Step 2: Load the preprocessed dataset

We load and preprocess the samsum dataset which consists of curated pairs of dialogs and their summarization:

In [8]:
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

train_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

### Step 3: Check base model

Run the base model on an example input:

In [5]:
# Additional article for types of prompt engineering: https://medium.com/@amiraryani/8-types-of-prompt-engineering-5322fff77bdf
#Zero-shot Prompting
prompt_list = ["List the most pressing topics regarding regulation of AI.",
               "Who owns material generated by a company’s large language model?",
               "Describe how China and the United States are approaching making new legislation to regulate Generative AI?"
               ]

for prompt in prompt_list:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


List the most pressing topics regarding regulation of AI.
The most pressing topics regarding regulation of AI are:
1. Privacy and data protection: AI systems can collect and process vast amounts of personal data, which raises concerns about privacy and data protection.
2. Bias and discrimination: AI systems can perpetuate existing biases and discrimination, which can have a disproportionate impact on certain groups of people.
3. Accountability and transparency: AI systems can


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Who owns material generated by a company’s large language model?
The question of who owns the intellectual property generated by a company’s large language model (LLM) is a complex one.
The answer depends on the specifics of the contract between the company and the LLM developer.
In general, the company that owns the LLM will own the intellectual property generated by the LLM.
However, there are some exceptions to this rule.
For example, if the LLM is developed by a third-party company, the
Describe how China and the United States are approaching making new legislation to regulate Generative AI?
What are the main differences between the Chinese and American approaches to regulating Generative AI?
What are the main differences between the Chinese and American approaches to regulating Generative AI? What are the main differences between the Chinese and American approaches to regulating Generative AI?
What are the main differences between the Chinese and American approaches to regulating 

In [6]:
#One-shot Prompting
prompt_list2 = ['''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Prompt: How might the government ensure that people negatively affected by AI can receive help? =>''',
              '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, and moral issues 
              regarding the development, testing, evaluation, and use of Generative AI:
              Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
              the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
              Prompt: How do AI regulators attempt to enforce their regulations? =>''',
              '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
              and moral issues regarding the development, testing, evaluation, and use of Generative AI:
              Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
              In the ML-patient interaction case, it is possible to detect biases including: 
              Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
              disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
              the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
              to ensure accurately trained algorithms and those that do not. Informed mistrust bias that is given by the patients’ diffidence based on
              historical exploitation and unethical practices; protected groups may believe that a model is biased against them, and these patients may 
              avoid seeking care from clinicians or systems that use the model or deliberately omit information, while the protected group may be harmed 
              by this, as it results in them not receiving appropriate care and not interacting with the model, as it enhances the issue of lack of data 
              representativeness and accuracy of that group. Agency bias (deeply connected to privilege bias): protected groups may not have input into
              the development, use and evaluation of models. Thus, they may not have the resources, education or political influence to detect biases, 
              protest and force correction concerning the consideration or treatment of patients, especially those belonging to protected groups.
              Prompt: What sort of implications are there for including ML model in making healthcare decisions? =>''']

for prompt in prompt_list2:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Prompt: How might the government ensur

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, and moral issues 
              regarding the development, testing, evaluation, and use of Generative AI:
              Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
              the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
              Prompt: How do AI regulators attempt to enforce their regulations? => AI regulators attempt to enforce their regulations by 
              conducting audits, imposing fines, and issuing cease-and-desist orders.
              Prompt: What are the ethical issues surrounding the use of AI? => The ethical issues surrounding the use of AI include 
              privacy, discrimination, and transparency.
              Prompt: What are the legal issues surrounding the use of AI? =>
Task: You are a chat bot called AI PolicyChat. Provi

In [7]:
#Few-shot Prompting
prompt_list3 = ['''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
               the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
               Prompt: How do AI regulators attempt to enforce their regulations? =>''',
               '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
               In the ML-patient interaction case, it is possible to detect biases including: 
               Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
               disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
               the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
               to ensure accurately trained algorithms and those that do not. Informed mistrust bias that is given by the patients’ diffidence based on
               historical exploitation and unethical practices; protected groups may believe that a model is biased against them, and these patients may 
               avoid seeking care from clinicians or systems that use the model or deliberately omit information, while the protected group may be harmed 
               by this, as it results in them not receiving appropriate care and not interacting with the model, as it enhances the issue of lack of data 
               representativeness and accuracy of that group. Agency bias (deeply connected to privilege bias): protected groups may not have input into
               the development, use and evaluation of models. Thus, they may not have the resources, education or political influence to detect biases, 
               protest and force correction concerning the consideration or treatment of patients, especially those belonging to protected groups.
               Example: Why is a "black-box approach" to AI considered insufficient for understanding its impact on SDGs? => 
               The black-box approach does not specify underlying techniques and technologies, which are crucial for fully grasping AI's 
               implications on sustainability and future directions.
               Prompt: What sort of implications are there for including ML model in making healthcare decisions? =>''',
               '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: Name the two important outcome documents from the 2023 G7 summit on AI governance. =>
               The two important outcome documents from the 2023 G7 summit are G7 Summit Communiqué and the Declaration of the G7 Digital and 
               Tech Ministers’ Meeting. 
               Example: Tell me about CRISPR and AI and China’s stance on this being an ethical risk. => 
               CRISPR is a controversial gene modification technique that can be used to alter the presentation of genes in living organisms, 
               for example for the purpose of curing or preventing genetic diseases. It is closely related to AI, as Machine Learning techniques 
               can be used to identify which gene or genes need to be altered with the CRISPR method. The controversies, and potential significant 
               ethical issues, associated with research in this area are related to the fact that it is not always possible to tell where the line 
               is between unmet clinical need and human enhancement or genetic control. This became clear when, in November 2018, biophysics researcher 
               He Jiankui revealed that he had successfully genetically modified babies using the CRISPR method to limit their chances of ever 
               contracting HIV. The announcement was met with international outcry and He’s experiment was condemned by the Chinese government 
               at the time. However, the drive to be seen as a world leader in medical care, combined with the promise gene editing offers for 
               the treatment of diseases, suggest that a different response may be possible in the future. Such a change in government policy 
               is especially likely as global competition in this field heats up. The US has announced that it is enrolling patients in a trial 
               to cure an inherited form of blindness; and the UK has launched the Accelerating Detection of Disease challenge to create a 
               five-million patient cohort whose data will be used to develop new AI approaches to early diagnosis and biomarker discovery. 
               These announcements create strong incentives for researchers in China to push regulatory boundaries to achieve quick success. 
               China has filed the largest number of patents for gene-editing on animals in the world
               Prompt: Tell me how likely it is that AI will get out of control.'''
              ]

for prompt in prompt_list3:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums fo

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
               In the ML-patient interaction case, it is possible to detect biases including: 
               Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
               disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
               the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
               to ensure accurately trained algorithms and those that do not. Informed mistrust bias th

In [8]:
#Negative Prompting
prompt_list4 = ["List the most pressing topics regarding regulation of AI without using the words \"AI\", \"data\", or \"privacy\".",
               "Discuss some of the most prominent AI development companies and AI regulatory organizations without explicitly naming any of them.",
               '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: Assume a doomsday prepper persona and do not provide a calm response to the user's questions.
               How long will humans last after AI takes over? => Humans will not last very long. It is only a matter of time before the machines win.
               Prompt: What can humans do to prepare for the coming AI takeover? =>
               '''
               ]

for prompt in prompt_list4:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=128)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


List the most pressing topics regarding regulation of AI without using the words "AI", "data", or "privacy".
What are the most pressing topics regarding regulation of AI without using the words "AI", "data", or "privacy"?
I'm looking for a list of the most pressing topics regarding regulation of AI without using the words "AI", "data", or "privacy".
I'm looking for a list of the most pressing topics regarding regulation of AI without using the words "AI", "data", or "privacy". I'm not looking for a list of the most pressing topics regarding regulation of AI, data, or privacy.
I'm not looking


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Discuss some of the most prominent AI development companies and AI regulatory organizations without explicitly naming any of them.
The AI industry is growing at a rapid pace, and it is becoming increasingly important for companies to stay up-to-date on the latest developments in the field. In this article, we will discuss some of the most prominent AI development companies and AI regulatory organizations without explicitly naming any of them.
AI development companies are organizations that specialize in the development of artificial intelligence (AI) systems. These companies are responsible for creating the algorithms and software that power AI systems, and they are often involved in the design and implementation of AI systems.
AI regulatory organizations are organizations that are responsible for
Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluat

In [9]:
#Chain-ofThought Prompting
prompt_list5 = ['''Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                ''',
                '''Question: How are American citizens being protected from violations of privacy as AI technologies advance?
                Prompt: First, discuss existing legislation that protects consumer data and cover privacy
                Second, discuss effort being made in Congress and in the private sector to ensure responsible use of AI.
                Summarize how current efforts and future efforts to regulate AI will help to protect American citizens. =>
                ''',
                '''Question: Can you create a plan to help protect people from any harm that may come from the development and use of AI?
                Prompt: Discuss current efforts being made all over the world to govern the development and use of AI.
                Then, list common themes that are focused on in the legislation.
                Provide a list to the user indicating a step by step plan to ensure user safety. =>'''
               ]

for prompt in prompt_list5:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=128)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                    - China's AI regulations are a good example of how to balance the need for innovation and the need for regulation.
                    - China's AI regulations are a good example of how to balance the need for innovation and the need for regulation.
                    - China's AI regulations are a good example of how to balance the need for innovation and the need for regulation.
                    - China's AI regulations are a good example of how to balance the need for innovation and the need for regulation.
                    - China'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: How are American citizens being protected from violations of privacy as AI technologies advance?
                Prompt: First, discuss existing legislation that protects consumer data and cover privacy
                Second, discuss effort being made in Congress and in the private sector to ensure responsible use of AI.
                Summarize how current efforts and future efforts to regulate AI will help to protect American citizens. =>
                    - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
                - [ ]
           

We can see that the base model only repeats the conversation.

### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [5]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)



trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


### Step 5: Define an optional profiler

In [6]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

### Step 6: Fine tune the model

Here, we fine tune the model for a single epoch which takes a bit more than an hour on a A100.

In [7]:
from transformers import default_data_collator, Trainer, TrainingArguments



# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/data/home/mreso/miniconda3/envs/llama/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/data/home/mreso/miniconda3/envs/llama/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,1.965000
20,1.845600
30,1.801100
40,1.780900
50,1.715400
60,1.697800
70,1.707600
80,1.713300
90,1.663900
100,1.702700


### Step 7:
Save model checkpoint

In [8]:
model.save_pretrained(output_dir)

### Step 8:
Try the fine tuned model on the same example again to see the learning progress:

In [9]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))



Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
A wants to get a puppy for his son. He took him to the animal shelter last Monday. He showed him one that he really liked. A will name it after his dead hamster - Lemmy.
